In [1]:
import pandas as pd
from Bio import Restriction, SeqIO
from ntaxon.fingerprinting.rflp import RestrictionDigestion
import os
import pandas as pd

In [2]:
combined_seq = pd.read_csv('./data/combined_seq.csv')
combined_seq.head()

,accession,species,sequence,remark,name
0,MN513225.1,Alcaligenes faecalis,ATTGAACGCTAGCGGGATGCTTTACACATGCAAGTCGAACGGCAGC...,R,R_MN513225.1
1,JF710959.1,Alcaligenes faecalis,TACACATGCAAGTCGAACGGCAGCACGAGAGAGCTTGCTCTCTTGG...,R,R_JF710959.1
2,KT988067.1,Alcaligenes faecalis,ATTGAACGCTAGCGGGATGCTTTACACATGCAAGTCGAACGGCAGC...,R,R_KT988067.1
3,KP224304.1,Alcaligenes faecalis,GAGTTTGATCCTGGCTCAGATTGAACGCTAGCGGGATGCTTTACAC...,R,R_KP224304.1
4,KF534470.1,Alcaligenes faecalis,GAGTTTGATCCTGGCTCAGATTGAACGCTAGCGGGATGCTTTACAC...,R,R_KF534470.1


In [3]:
sample_seq = combined_seq[combined_seq['remark'] == 'S']
sample_seq.head()

,accession,species,sequence,remark,name
203,MN493874.1,Stenotrophomonas maltophilia,AAGGGGTGGCCTACACATGCAAGTCGAACGGCAGCACAGGAGAGCT...,S,S_MN493874.1
204,MN493875.1,Brevundimonas naejangsanensis,GGCGCAGGCCTACACATGCAAGTCGAACGAACTCTTCGGAGTTAGT...,S,S_MN493875.1
205,MN493876.1,Stenotrophomonas pavanii,AATGCGGGGCCTACACATGCAAGTCGAACGGCAGCACAGGAGAGCT...,S,S_MN493876.1
206,MN493877.1,Ochrobactrum anthropi,CCAAGGGCGGCCTTACCATGCAAGTCGAGCGCCCCGCAAGGGGAGC...,S,S_MN493877.1
207,MN493878.1,Ochrobactrum anthropi,GCAGCTTACCATGCAAGTCGAGCGCCCCGCAAGGGGAGCGGCAGAC...,S,S_MN493878.1


# Phylotype screening for Restriction Enzyme

In [77]:
#enzyme = Restriction.AluI
#hap_prefix = "AL"

#enzyme = Restriction.HpaII
#hap_prefix = "HP"

enzyme = Restriction.MspI
hap_prefix = "MS"

#enzyme = Restriction.RsaI
#hap_prefix = "RS"

In [78]:
haplotypes = pd.read_csv(f"./outputs/haplotypes_sp_{enzyme.__name__}.csv")
haplotypes['haplotypes'] = haplotypes['haplotypes'].apply(lambda x: eval(x))
haplotypes['haplotype_sizes'] = haplotypes['haplotype_sizes'].apply(lambda x: eval(x))
haplotypes.drop(haplotypes.columns[0], axis=1, inplace=True) # drop first index column
haplotypes.head()

,species,haplotypes,haplotype_sizes
0,Achromobacter marplatensis,"[118_1, 130_1, 81_1]","[118, 130, 81]"
1,Acinetobacter rudis,"[799_1, 81_1]","[799, 81]"
2,Advenella kashmirensis,"[122_1, 130_1, 81_1]","[122, 130, 81]"
3,Alcaligenes faecalis,"[118_1, 81_1]","[118, 81]"
4,Bacillus aerius,"[211_1, 391_1, 538_1, 60_1]","[211, 391, 538, 60]"


In [79]:
# create a haplotype identity column and delete duplicate
haplotypes['uid'] = haplotypes['haplotypes'].apply(lambda x: "-".join(x))
haplotypes = haplotypes.drop_duplicates(subset=['uid'], keep="last")
haplotypes['fragment_count'] = haplotypes['haplotypes'].apply(lambda x: len(x))
haplotypes.head()

,species,haplotypes,haplotype_sizes,uid,fragment_count
0,Achromobacter marplatensis,"[118_1, 130_1, 81_1]","[118, 130, 81]",118_1-130_1-81_1,3
1,Acinetobacter rudis,"[799_1, 81_1]","[799, 81]",799_1-81_1,2
2,Advenella kashmirensis,"[122_1, 130_1, 81_1]","[122, 130, 81]",122_1-130_1-81_1,3
3,Alcaligenes faecalis,"[118_1, 81_1]","[118, 81]",118_1-81_1,2
4,Bacillus aerius,"[211_1, 391_1, 538_1, 60_1]","[211, 391, 538, 60]",211_1-391_1-538_1-60_1,4


In [80]:
all_digestion = RestrictionDigestion(
    accessions=combined_seq, 
    enzyme=enzyme, 
    label_col="name",
    sequence_col="sequence"
)
all_matrix_bin = all_digestion.binary_matrix

samp_digestion = RestrictionDigestion(
    accessions=sample_seq, 
    enzyme=enzyme, 
    label_col="name",
    sequence_col="sequence"
)
samp_matrix_bin = samp_digestion.binary_matrix

# Search for matched Identifiers
Identifiers may also be from another species

In [81]:
def get_all_haplotypes(x):
    try:
        return all_matrix_bin.filter_by_fragment(fragments=x).sample_names
    except:
        return []

def get_sample_haplotypes(x):
    try:
        return samp_matrix_bin.filter_by_fragment(fragments=x).sample_names
    except:
        return []

haplotypes['all_identifiers'] = haplotypes['haplotypes'].apply(get_all_haplotypes)
haplotypes['sample_identifiers'] = haplotypes['haplotypes'].apply(get_sample_haplotypes)
haplotypes.head()

,species,haplotypes,haplotype_sizes,uid,fragment_count,all_identifiers,sample_identifiers
0,Achromobacter marplatensis,"[118_1, 130_1, 81_1]","[118, 130, 81]",118_1-130_1-81_1,3,"[R_JF710959.1, R_KF534470.1, R_KP224304.1, R_K...","[S_MN493881.1, S_MN493882.1, S_MN493886.1, S_M..."
1,Acinetobacter rudis,"[799_1, 81_1]","[799, 81]",799_1-81_1,2,"[R_AB859674.1, R_AB859737.1, R_FN298236.1, R_K...",[S_MN577382.1]
2,Advenella kashmirensis,"[122_1, 130_1, 81_1]","[122, 130, 81]",122_1-130_1-81_1,3,"[R_KF956701.1, R_LN870300.1, R_MH379789.1, R_M...",[S_MN577386.1]
3,Alcaligenes faecalis,"[118_1, 81_1]","[118, 81]",118_1-81_1,2,"[R_JF710959.1, R_KF534470.1, R_KP224304.1, R_K...","[S_MN493881.1, S_MN493882.1, S_MN493886.1, S_M..."
4,Bacillus aerius,"[211_1, 391_1, 538_1, 60_1]","[211, 391, 538, 60]",211_1-391_1-538_1-60_1,4,"[R_KT441039.1, R_KU358906.1, R_NR_042338.1]",[]


In [82]:
haplotypes.to_csv(f"./outputs/haplotypes_sp_{enzyme.__name__}_identified.csv")

In [83]:
haplotypes_filt = haplotypes[haplotypes['fragment_count'] > 1]
haplotypes_filt['id'] = haplotypes_filt.index + 1

<ipython-input-83-76d2564321f5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  haplotypes_filt['id'] = haplotypes_filt.index + 1


In [84]:
haps_flat = pd.DataFrame(columns=['hap_id', 'hap', 'sample', 'accession'])
# drop blank
for idx, r in haplotypes_filt[['id', 'sample_identifiers']].iterrows():
    for s in r['sample_identifiers']:
        haps_flat = haps_flat.append({
            'hap_id': r['id'], 
            'hap': f"{hap_prefix}{r['id']}", 
            'sample': s, 
            'accession': s.split('_')[1]
        }, ignore_index=True)
haps_flat.tail()

,hap_id,hap,sample,accession
94,36,MS36,S_MN493876.1,MN493876.1
95,36,MS36,S_MN493907.1,MN493907.1
96,36,MS36,S_MN493909.1,MN493909.1
97,36,MS36,S_MN577375.1,MN577375.1
98,37,MS37,S_MN577385.1,MN577385.1


In [85]:
haps_flat.to_csv(f"./outputs/haplotypes_accessions_{enzyme.__name__}.csv")